In [ ]:
from cedne import utils

In [ ]:
## Make a worm
w = utils.makeWorm()
nn = w.networks["Neutral"]

## Adding some properties to the neurons (see Example 1)
num_sensory = len([_ for n in nn.neurons if nn.neurons[n].type == 'sensory'])
num_motor = len([_ for n in nn.neurons if nn.neurons[n].category == 'head motor neuron'])
for n in nn.neurons:
    if nn.neurons[n].type == 'sensory':
        nn.neurons[n].set_property('demand', -num_motor)
    #elif nn.neurons[n].type == 'motorneuron':
    elif nn.neurons[n].category == 'head motor neuron':
        nn.neurons[n].set_property('demand', num_sensory)
    else:
        nn.neurons[n].set_property('demand', 0) 

for e, conn in nn.connections.items():
    conn.set_property('capacity', conn.weight)

In [ ]:
total_demand = sum(nn.neurons[n].get_property('demand') for n in nn.neurons)
print(total_demand)

In [ ]:
disconnected = []
for sensory in [nn.neurons[n] for n in nn.neurons if nn.neurons[n].type == "sensory"]:
    if not any(utils.nx.has_path(nn, sensory, motor) for motor in [nn.neurons[n] for n in nn.neurons if nn.neurons[n].category == "head motor neuron"]):
        disconnected.append(sensory)

In [ ]:
print([n for n in nn.neurons if nn.neurons[n].category in ["head motor neuron"]])

In [ ]:
def depth_first_search(source, target, direction='reverse', max_length=7):
    """ Traverse the graph depth-first and write out the set of all paths between source and target neurons. 
    Traversal ends at a maximum of max_length steps."""

    source_n = nn.neurons[source]
    target_n = nn.neurons[target]

    def dfs(current, target, path, visited, paths, max_length):
        if current == target:
            paths.append(path)
            return
        if len(path) > max_length:
            return
        visited.add(current)
        if direction == 'reverse':
            for eid, conn in current.get_connections(direction='in').items():
                if eid[0] not in visited:
                    dfs(eid[0], target, path + [eid[0]], visited, paths, max_length)
        elif direction == 'forward':
            for eid, conn in current.get_connections(direction='out').items():
                if eid[1] not in visited:
                    dfs(eid[1], target, path + [eid[1]], visited, paths, max_length)
        visited.remove(current)

    paths = []
    dfs(source_n, target_n, [source_n], set(), paths, max_length)
    return paths


In [ ]:
depth_first_search('RIAL', 'AWCL')

In [ ]:
paths